In [1]:
import ee
import geemap
import geopandas as gpd
import json

service_account = 'ee-account@airy-galaxy-398310.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '.private-key.json')

ee.Initialize(credentials)

import classification as CF
import accuracy as ACC
from add_column import add_landcover_column

In [2]:
datasets = [
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS",
    "projects/sat-io/open-datasets/NINA/ELC10",
    "COPERNICUS/CORINE/V20/100m"
]

In [3]:
# Find the valid clasees and band name to classify
print(ee.ImageCollection(datasets[1]).first().bandNames().getInfo())
print(ee.ImageCollection(datasets[1]).first().getInfo())

from_list = [1,2,4,5,7,8,9,10,11]
to_list = [i for i in range(9)]

['b1']
{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [20686, 89414], 'crs': 'EPSG:32701', 'crs_transform': [10, 0, 396569.58269600547, 0, -10, 2011064.3450437225]}], 'version': 1663385245823962, 'id': 'projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS/01C_20170101-20180101', 'properties': {'system:time_start': 1483234200000, 'num_bands': 1, 'id_no': '01C_20170101-20180101', 'ysize': 89414, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-172.72352926547543, -77.47593585421382], [-173.03460177536232, -76.47695983709995], [-173.30249658490283, -75.47755394033669], [-173.5355377617104, -74.47778004397523], [-173.7400397987115, -73.47768411126914], [-174.003669971275, -71.97691915929559], [-179.99635415698984, -71.97691871038212], [179.74001347125807, -73.47768372626119], [179.53550980159105, -74.47777956454652], [179.3024668571339, -75.47755343070565], [179.0345697832042, -76.4

In [4]:
image = CF.classification(datasets[0])

# If there is remapping, new column is "remapped"
# image = image.remap(from_list, to_list, defaultValue=0, bandName='landcover')

In [5]:
table = ACC.get_acc(image, params={
    'scale': 30
})

CF.export_table_task(table)

EEException: Computation timed out.

In [ ]:
filename = "input/input0.geojson"
gdf = add_landcover_column(**{
    'ee_image': image,
    'gpd_file': filename,
    'classes_qnt': 9,
    'pixel_key': 'b1',
    'column_name': 'landcover'
})

output_file = "output/output0_ESRI10m.geojson"
gdf.to_file(output_file, driver="GeoJSON")